# **Importamos la librerias necesarias**

In [1]:
from _utils import *

# **Ahora cargamos los dataset necesarios**

In [25]:
df = pd.DataFrame(read_json("../../datasets/json/coins.json"))

# **Inspeccion de Datos**


Ahora le damos una vista general de los datos y se limpiaran los datos 

`df_coins`

In [26]:
df.head(5)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,62485.000000,1.234314e+12,1.0,1.316328e+12,1.745832e+10,64258.000,...,2024-03-14T07:10:36.635Z,67.810000,9.233951e+04,2013-07-06T00:00:00.000Z,None,2024-04-29T03:07:50.739Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...",[thorswap]
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,3204.580000,3.933157e+11,2.0,3.933157e+11,1.234581e+10,3345.290,...,2021-11-10T14:24:19.604Z,0.432979,7.444012e+05,2015-10-20T00:00:00.000Z,"{'times': 67.51867213370284, 'currency': 'btc'...",2024-04-29T03:07:34.938Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...","[curve_ethereum, thorswap, openocean_finance]"
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999644,1.104947e+11,3.0,1.104947e+11,2.619696e+10,1.001,...,2018-07-24T00:00:00.000Z,0.572521,7.468564e+01,2015-03-02T00:00:00.000Z,None,2024-04-29T03:05:41.166Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[jupiter, raydium2, curve_ethereum, meteora, t..."
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,592.640000,9.147265e+10,4.0,9.147265e+10,7.763166e+08,606.920,...,2021-05-10T07:24:17.097Z,0.039818,1.493039e+06,2017-10-19T00:00:00.000Z,None,2024-04-29T03:07:39.939Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[uniswap_v3, thorswap, openocean_finance]"
4,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,135.710000,6.090000e+10,5.0,7.827803e+10,2.231952e+09,144.580,...,2021-11-06T21:54:35.825Z,0.500801,2.710871e+04,2020-05-11T19:35:23.449Z,None,2024-04-29T03:07:43.805Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...",[]


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7575 entries, 0 to 7574
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                7575 non-null   object 
 1   symbol                            7575 non-null   object 
 2   name                              7575 non-null   object 
 3   image                             7575 non-null   object 
 4   current_price                     7573 non-null   float64
 5   market_cap                        7574 non-null   float64
 6   market_cap_rank                   3622 non-null   float64
 7   fully_diluted_valuation           6961 non-null   float64
 8   total_volume                      7574 non-null   float64
 9   high_24h                          7321 non-null   float64
 10  low_24h                           7321 non-null   float64
 11  price_change_24h                  7321 non-null   float64
 12  price_

Observamos la frecuencia de las exchanges que pertenece cada moneda

In [20]:
# Cantidad de exchanges centralizados (frecuencia)
df["exchanges_centralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_centralized
0    3376
1    2166
2     608
3     316
4     206
5     113
6     110
9      66
7      65
8      58
Name: count, dtype: int64

In [21]:
# Cantidad de exchanges descentralizados (frecuencia)
df["exchanges_decentralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_decentralized
1    3193
0    2438
2    1110
3     428
4     225
5     102
6      34
9      11
8      11
7      10
Name: count, dtype: int64

## Caso especial

Antes de realizar la limpieza de los datos, podemos observar los siguiente del `JSON` del dataset:
- La exchange a la cual pertenece es una version de esta pasada `uniswap_v2`, ya que actualmente esta la version 3.
- La criptomoneda no ha tenido movimiento las ultimas 24 horas.
- No tiene maximos ni minimos historicos.
  
Por lo cual podemos deducir que se debe a un error en el registro de la moneda o un problema con la version de la exchange de donde la API nos proporciono esta información 

```json
{
	"id": "party",
	"symbol": "party",
	"name": "PARTY",
	"image": "https://assets.coingecko.com/coins/images/33550/large/party-logo-200x200.png?1702397278",
	"current_price": null,
	"market_cap": null,
	"market_cap_rank": null,
	"fully_diluted_valuation": null,
	"total_volume": null,
	"high_24h": null,
	"low_24h": null,
	"price_change_24h": null,
	"price_change_percentage_24h": null,
	"market_cap_change_24h": null,
	"market_cap_change_percentage_24h": null,
	"circulating_supply": null,
	"total_supply": 420177600000000.0,
	"max_supply": 420177600000000.0,
	"ath": null,
	"ath_change_percentage": 0.0,
	"ath_date": null,
	"atl": null,
	"atl_change_percentage": 0.0,
	"atl_date": null,
	"roi": null,
	"last_updated": null,
	"category": "Pixels Game",
	"category_id": "pixels-game",
	"exchanges_centralized": [],
	"exchanges_decentralized": [
		"uniswap_v2"
	]
}
```

Este caso especial se tendra que eliminar del dataset 

In [28]:
# Eliminar la fila con id "party"
df = df[df["id"] != "party"]

In [30]:
# Validamos que no existe ese json
df[df["id"] == "party"]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized


# **Preprocesamiento de los Datos**

In [22]:
find_columns_with_nulls(df)

[('current_price', 2),
 ('market_cap', 1),
 ('market_cap_rank', 3953),
 ('fully_diluted_valuation', 614),
 ('total_volume', 1),
 ('high_24h', 254),
 ('low_24h', 254),
 ('price_change_24h', 254),
 ('price_change_percentage_24h', 254),
 ('market_cap_change_24h', 256),
 ('market_cap_change_percentage_24h', 256),
 ('circulating_supply', 1),
 ('total_supply', 505),
 ('max_supply', 2278),
 ('ath', 1),
 ('ath_date', 1),
 ('atl', 1),
 ('atl_date', 1),
 ('roi', 7302),
 ('last_updated', 1)]